In [150]:
import numpy as np
import matplotlib.pyplot as plt
import torch
plt.style.use('dark_background')

In [161]:
L_opt = 300*(1e-12)
w = 2*np.pi*(20e9)
Z_L = 20+1j*(w*L_opt) # (target) load impedance
Z_S = 20 # fixed source impedance

def computeGamma(Z,Z0=50): z = Z/Z0; return ((z-1)/(z+1))
def transformedImpedance(match__Lout_pH):#,match__Cout):
#     # Series inductor, Shunt capacitor (Source --> Load)
#     # A = 1+ind_x*cap_b; B = ind_x; C = cap_b; D = 1
#     # Z_S_transformed = (D*Z_S + B)/(C*Z_S+A)
    
    match__Cout_fF = 0
    cap_b = 1j*w*match__Cout_fF*(1e-15)
    ind_x = 1j*w*match__Lout_pH*(1e-12)
    A = 1 + ind_x*cap_b; B = ind_x; C = cap_b; D=1
    Z_S_transformed = (D*Z_S+B)/(C*Z_S+A)
    # Z_S_transformed = Z_S + 1j*w*match__Lout_pH*(1e-12)
    return Z_S_transformed

def computeError(match__Lout_pH):#,match__Cout):
    # These work
    # impedance_error = torch.abs(match__Lout_pH - 300)**2
    # impedance_error = torch.abs(w*match__Lout_pH - w*300)**2
    
    # Z_S_transformed = Z_S + 1j*w*match__Lout_pH
    Z_S_transformed = transformedImpedance(match__Lout_pH)
    impedance_error = torch.abs(Z_S_transformed - Z_L)**2

    return impedance_error

In [162]:
match__Lout = torch.tensor(10.9, requires_grad=True)

optimizer = torch.optim.Adam([match__Lout], lr=10) # Adaptive algorithm, better
for i in range(500):
    optimizer.zero_grad()
    f = computeError(match__Lout)
    f.backward()
    optimizer.step()
    if i % 100 == 0:
        print(f":Iteration {i}:  Lout = {match__Lout.item()}, error = {f.item()}")

:Iteration 0:  Lout = 20.89999771118164, error = 1319.8236083984375
:Iteration 100:  Lout = 298.51043701171875, error = 0.043113451451063156
:Iteration 200:  Lout = 299.997314453125, error = 4.401954356580973e-08
:Iteration 300:  Lout = 300.0000915527344, error = 5.820766091346741e-11
:Iteration 400:  Lout = 300.0000915527344, error = 5.820766091346741e-11
